## _Building True Edges_

1. _~~layerwise true edges~~_
2. _~~modulewise true edges~~_
3. _orderwise true edges (new for curly tracks)_

In [ ]:
import glob, os, sys, yaml

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pprint
import seaborn as sns
import trackml.dataset

In [ ]:
import torch
from torch_geometric.data import Data
import itertools

In [ ]:
# append parent dir
sys.path.append('..')

In [ ]:
# get cuda gpus if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# local imports
from src import SttCSVDataReader, SttTorchDataReader
from src import detector_layout
from src import Build_Event, Build_Event_Viz, Visualize_Edges
from src.math_utils import polar_to_cartesian

### _Input Data_

In [ ]:
# input data
input_dir = '../data_all'

In [ ]:
# Find All Input Data Files (hits.csv, cells.csv, particles.csv, truth.csv)
all_files = os.listdir(input_dir)

# Extract File Prefixes (use e.g. xxx-hits.csv)
suffix = '-hits.csv'
file_prefixes = sorted(os.path.join(input_dir, f.replace(suffix, '')) 
                       for f in all_files if f.endswith(suffix))

print("Number of Files: ", len(file_prefixes))

In [ ]:
# file_prefixes[:10]

In [ ]:
# load an event
# hits, tubes, particles, truth = trackml.dataset.load_event(file_prefixes[0])

In [ ]:
# hits.head()
# tubes.head()
# particles.head()
# truth.head()

### _Visualize Event_

In [ ]:
# select event
event_id = 95191

In [ ]:
# compose event is exactly the same as select_hits()
# event = Build_Event(input_dir, event_id, noise=False, skewed=False, selection=False)

In [ ]:
# visualize event
# Build_Event_Viz(event, figsize=(10,10), fig_type="pdf", save_fig=False)

## _3. Orderwise True Edges_

Ground truth constructed from _`get_layerwise_edges()`_ and _`get_modulewise_edges()`_ heuristics works best for high $p_t$ particle trajectories _i.e._ ones that leave the detector. However, both of these methods fails when a low $p_t$ particle either re-enters the detector or simply spiral inside the detector. One needs a new heuristic for such tracks. Instead of sorting hits according to $R = \sqrt(x^2 + y^2 + z^2)$, one needs something that follows the praticle trajectory to create the grouth truth. There different ideas who to achieve that:

- By using a time parameter to sort hits in a track and connecting in order. Effectively hits will get connected along the path. Although timing is available during simulation but it get disturbed during digitization.
- Although timing parmeter can not be used but we can assume there is inherent order, after all, hits are produced along the track sequentially during the simulation. One needs to make sure, any manipulation on the data later one does not disturb the order.

Another way is to sort the hits based on $\phi$ and $\theta$ in addition to $R$. In a magnetic field, a particle either bends left of right based on its charge. In principle, one can sort hits using the $\phi$ in addition to $R$. Alternatively, one make use of direction cosines to sort hits along the particle trajectory.

Lastly, one can connect hits by using some sort of Nearest-Neighbor method.

___

### _3.1 - Order of Occurence of Hits [Success]_

We assume there is an inherent order in the hit position of a particle trajectory. This is confimed when looking into the timing of hits during _simulation_. So in principle one can use timing to keep track the order. However, timing might not be useful after digitization but it should not disturbed the order. Now we check while processing an event this order is not disturbed by different operations on the DataFrames. We with investigate `select_hits()` where a lot of processing is happening.

In [ ]:
def process_particles(particles, selection=False):
    """Special manipulation on particles dataframe"""
    
    # find nhits, and drop_duplicates
    particles['nhits'] = particles.groupby(['particle_id'])['nhits'].transform('count')
    particles.drop_duplicates(inplace=True, ignore_index=True)
    
    if selection:
        # just keep protons, pions, don't forget resetting index and dropping old one.
        particles = particles[particles['pdgcode'].isin([-2212, 2212, -211, 211])].reset_index(drop=True)
    
    return particles

In [ ]:
def select_hits(event_file=None, noise=False, skewed=False, **kwargs):
    """Hit selection method from Exa.TrkX. Build a full event, select hits based on certain criteria."""
    
    # load data using event_prefix (e.g. path/to/event0000000001)
    hits, tubes, particles, truth = trackml.dataset.load_event(event_file)

    # FIXME: Add an index column to preserve the original order of hits
    hits['original_order'] = hits.index

    # preprocess particles dataframe e.g. nhits, drop_duplicates, etc.
    particles = process_particles(particles, selection=kwargs['selection'])

    # skip noise hits.
    if noise:
        # runs if noise=True
        truth = truth.merge(
            # particles[["particle_id", "vx", "vy", "vz"]], on="particle_id", how="left"
            particles[["particle_id", "vx", "vy", "vz", "q", "pdgcode"]], on="particle_id", how="left"
        )
    else:
        # runs if noise=False
        truth = truth.merge(
            # particles[["particle_id", "vx", "vy", "vz"]], on="particle_id", how="inner"
            particles[["particle_id", "vx", "vy", "vz", "q", "pdgcode"]], on="particle_id", how="inner"
        )

    # derive new quantities from truth
    px = truth.tpx
    py = truth.tpy
    pz = truth.tpz

    # calculate pt, ptheta, peta, pphi
    pt = np.sqrt(px**2 + py**2)
    ptheta = np.arctan2(pt, pz)             # OR, np.arccos(pz/p)
    peta = -np.log(np.tan(0.5 * ptheta))
    pphi = np.arctan2(py, px)

    # assign pt, ptheta, peta, pphi to truth
    truth = truth.assign(pt=pt, ptheta=ptheta, peta=peta, pphi=pphi)
    
    # FIXME: Check if Order is Changed
    assert (hits['original_order'] == hits.index).all(), "Order disturbed after merging with tubes"
    
    # merge some columns of tubes to the hits, I need isochrone, skewed & sector_id
    hits = hits.merge(tubes[["hit_id", "isochrone", "skewed", "sector_id"]], on="hit_id")

    # FIXME: Check if Order is Changed
    assert (hits['original_order'] == hits.index).all(), "Order disturbed after merging with tubes"

    # skip skewed tubes
    if skewed is False:
        print("Removing Skewed Layers...")
        # filter non-skewed layers (skewed==0 for non-skewed layers & skewed==1 for skewed layers)
        hits = hits.query('skewed==0')

        # rename layer_ids from 0,1,2...,17 & assign a new colmn named "layer"
        vlids = hits.layer_id.unique()
        n_det_layers = len(vlids)
        vlid_groups = hits.groupby(['layer_id'])
        hits = pd.concat([vlid_groups.get_group(vlids[i]).assign(layer_id=i) for i in range(n_det_layers)])

    
    # FIXME: Check if Order is Changed
    assert (hits['original_order'] == hits.index).all(), "Order disturbed after removing skewed tubes"

    # Calculate derived variables from 'hits'
    r = np.sqrt(hits.x**2 + hits.y**2)
    phi = np.arctan2(hits.y, hits.x)
    r3 = np.sqrt(hits.x**2 + hits.y**2 + hits.z**2)
    theta = np.arccos(hits.z / r3)
    eta = -np.log(np.tan(theta / 2.))

    # Merge 'hits' with 'truth', but first add r, phi, theta, eta
    hits = hits.assign(r=r, phi=phi, theta=theta, eta=eta).merge(truth, on="hit_id")
    
    # FIXME: Check if Order is Changed
    # assert (hits['original_order'] == hits.index).all(), "Order disturbed after merging with truth"
    
    # FIXME: Restore the original order of hits using the 'original_order' column
    hits = hits.sort_values(by='original_order').reset_index(drop=True)

    # Add 'event_id' column to this event.
    hits = hits.assign(event_id=int(event_file[-10:]))
    
    # FIXME: Drop the original_order column as it is no longer needed
    hits = hits.drop(columns=['original_order'])

    return hits

### _1st Attempt_

- assuming the hits are in the order of occurence.

In [ ]:
def get_orderwise_edges(hits, column='hit_id'):
    """The function closely resembles get_modulewise_edges(), one
    can introduce layerwise variant similar to get_layerwise_edges"""
    
    # Group by particle_id, similar to modulewise edges
    groups = hits.groupby(["particle_id"])

    # Create an empty list to store the edge arrays for each group
    true_edges = []

    # Iterate over each group
    for _, group in groups:
        
        # Use 'hit_id' column to create true_edges, I assume order 
        # of occurence of hits is along the particle trajectory.
        # hit_indices = group['hit_id'].values
        
        # Or, use index of hits to create true_edges, I assume order 
        # of occurence of hits is along the particle trajectory [KEEP it].
        hit_indices = group.index.values
        
        # Create arrays for source and target nodes
        source_nodes = hit_indices[:-1]
        target_nodes = hit_indices[1:]

        # Concatenate the source and target arrays vertically
        edge_array = np.column_stack((source_nodes, target_nodes))

        # Append the edge array to the list
        true_edges.append(edge_array)

    # Concatenate for all particle groups vertically
    true_edges = np.vstack(true_edges)
    return true_edges.T

In [ ]:
# get event prefix using event_id
event_prefix = file_prefixes[event_id]

In [ ]:
# select hits
kwargs = {"selection": False}
hits = select_hits(event_file=event_prefix, noise=False, skewed=True, **kwargs)

In [ ]:
# get true edges
true_edges = get_orderwise_edges(hits)

In [ ]:
# new (1): plotting true edges

# detector layout
fig, ax = detector_layout(figsize=(10,10))

# particle tracks
pids = np.unique(hits.particle_id)
for pid in pids:
    idx = hits.particle_id == pid
    ax.scatter(hits[idx].x.values, hits[idx].y.values, label='particle_id: %d' %pid)
    
# loop over source and target nodes
# for i, (source_node, target_node) in enumerate(true_edges.T):
for (source_node, target_node) in true_edges.T:
    source_pos = hits.loc[source_node, ['x', 'y']].values
    target_pos = hits.loc[target_node, ['x', 'y']].values
    ax.plot([source_pos[0], target_pos[0]], [source_pos[1], target_pos[1]], 'k-', linewidth=0.5)

# axis params
ax.legend(fontsize=12, loc='best')
fig.tight_layout()
# fig.savefig("orderwise_true_edges.pdf")

### _2nd Attempt_

- following the logic of _`get_modulewise_edges()`_

In [ ]:
def get_modulewise_edges_ordered (hits):
    """Get modulewise (layerless) true edge list using the order
    of occurence hits. Here 'hits' represent complete event."""
    
    # Handle NaN and Null Values    
    signal = hits[
        ((~hits.particle_id.isna()) & (hits.particle_id != 0)) & (~hits.vx.isna())
    ]
    signal = signal.drop_duplicates(
        subset=["particle_id", "volume_id", "layer_id", "module_id"]
    )
    
    # Reset index to get 'index' column
    signal = signal.reset_index(drop=False)
    
    # Preserve the Order
    # signal = signal.rename(columns={"index": "unsorted_index"}).reset_index(drop=False)

    # Handle Particle_id 0
    signal.loc[signal["particle_id"] == 0, "particle_id"] = np.nan
        
    # Group by particle ID and get list of indices of every particle (series of series).
    signal_list = signal.groupby(["particle_id"], sort=False)["index"].agg(
        lambda x: list(x)
    )
        
    # Generate Edges
    true_edges = []
    for row in signal_list.values:
        for i, j in zip(row[:-1], row[1:]):
            true_edges.append([i, j])
        
    # Return Edges
    true_edges = np.array(true_edges).T
    
    # Restore the Order
    # true_edges = signal.unsorted_index.values[true_edges]

    return true_edges

In [ ]:
# select hits
kwargs = {"selection": False}
hits = select_hits(event_file=event_prefix, noise=False, skewed=True, **kwargs)

In [ ]:
# get true edges
true_edges = get_modulewise_edges_ordered(hits)

In [ ]:
# new (2): plotting true edges

# detector layout
fig, ax = detector_layout(figsize=(10,10))

# particle tracks
hits_grouped = hits.groupby('particle_id')
for particle_id, group in hits_grouped:
    ax.scatter(group['x'], group['y'], label=f'particle_id: {particle_id}')

# loop over source and target nodes
# for i, (source_node, target_node) in enumerate(true_edges.T):
for (source_node, target_node) in true_edges.T:
    source_pos = hits.loc[source_node, ['x', 'y']].values
    target_pos = hits.loc[target_node, ['x', 'y']].values
    ax.plot([source_pos[0], target_pos[0]], [source_pos[1], target_pos[1]], 'k-', linewidth=0.5)

# axis params
ax.legend(fontsize=12, loc='best')
fig.tight_layout()
# fig.savefig("modulewise_edges_ordered.pdf")

### _3rd Attempt [Failed]_

- following the logic of _`get_layerwise_edges()`_

In [ ]:
def get_layerwise_edges_ordered (hits):
    """Get modulewise (layerless) true edge list using the order
    of occurence hits. Here 'hits' represent complete event."""
    
    hits = hits.reset_index()
    
    hits.loc[hits["particle_id"] == 0, "particle_id"] = np.nan
    hit_list = (
        hits.groupby(["particle_id", "layer_id"], sort=False)["index"]  # ADAK: layer >> layer_id
        .agg(lambda x: list(x))
        .groupby(level=0)
        .agg(lambda x: list(x))
    )

    true_edges = []
    for row in hit_list.values:
        for i, j in zip(row[0:-1], row[1:]):
            true_edges.extend(list(itertools.product(i, j)))
            
    true_edges = np.array(true_edges).T
    return true_edges, hits

In [ ]:
# select hits
kwargs = {"selection": False}
hits = select_hits(event_file=event_prefix, noise=False, skewed=True, **kwargs)

In [ ]:
# get true edges
true_edges, hit = get_layerwise_edges_ordered(hits)

In [ ]:
# new (2): plotting true edges

# detector layout
fig, ax = detector_layout(figsize=(10,10))

# particle tracks
hits_grouped = hits.groupby('particle_id')
for particle_id, group in hits_grouped:
    ax.scatter(group['x'], group['y'], label=f'particle_id: {particle_id}')

# loop over source and target nodes
# for i, (source_node, target_node) in enumerate(true_edges.T):
for (source_node, target_node) in true_edges.T:
    source_pos = hits.loc[source_node, ['x', 'y']].values
    target_pos = hits.loc[target_node, ['x', 'y']].values
    ax.plot([source_pos[0], target_pos[0]], [source_pos[1], target_pos[1]], 'k-', linewidth=0.5)

# axis params
ax.legend(fontsize=12, loc='best')
fig.tight_layout()
# fig.savefig("layerwise_edges_ordered.pdf")

___
### _3.2 - Angle-based Sorting of Hits [Failed]_

There are several reasons why this methods fails _e.g._ two hits in the same layer, uncertanity in the hit position as successive layers have hits that are not aligned within the precision of $\phi$.

In [ ]:
def get_modulewise_edges_with_angles(hits):
    """Get modulewise (layerless) true edge list, considering angular sorting."""
    # Filter out the hits that correspond to actual particles
    signal = hits[
        ((~hits.particle_id.isna()) & (hits.particle_id != 0)) & (~hits.vx.isna())
    ]
    signal = signal.drop_duplicates(
        subset=["particle_id", "volume_id", "layer_id", "module_id"]
    )

    # Compute the radial distance R and angles phi and theta
    signal = signal.assign(
        R=np.sqrt(
            (signal.x - signal.vx) ** 2
            + (signal.y - signal.vy) ** 2
            + (signal.z - signal.vz) ** 2
        )
    )

    # Group by particle ID
    signal_list = signal.groupby("particle_id", sort=False)

    true_edges = []
    for pid, group in signal_list:
        # Sort by R and then by phi, theta to handle re-entry
        group = group.sort_values(["R", "pphi", "ptheta"]).reset_index(drop=False)

        # Handle re-indexing
        group = group.rename(columns={"index": "unsorted_index"}).reset_index(drop=False)
        
        row_indices = group.index.values
        for i, j in zip(row_indices[:-1], row_indices[1:]):
            true_edges.append([group.loc[i, "unsorted_index"], group.loc[j, "unsorted_index"]])

    true_edges = np.array(true_edges).T
    return true_edges


In [ ]:
# select hits
kwargs = {"selection": False}
hits = select_hits(event_file=event_prefix, noise=False, skewed=False, **kwargs)

In [ ]:
true_edges = get_modulewise_edges_with_angles(hits)

In [ ]:
# new (2): plotting true edges

# detector layout
fig, ax = detector_layout(figsize=(10,10))

# particle tracks
hits_grouped = hits.groupby('particle_id')
for particle_id, group in hits_grouped:
    ax.scatter(group['x'], group['y'], label=f'particle_id: {particle_id}')

# loop over source and target nodes
# for i, (source_node, target_node) in enumerate(true_edges.T):
for (source_node, target_node) in true_edges.T:
    source_pos = hits.loc[source_node, ['x', 'y']].values
    target_pos = hits.loc[target_node, ['x', 'y']].values
    ax.plot([source_pos[0], target_pos[0]], [source_pos[1], target_pos[1]], 'k-', linewidth=0.5)

# axis params
ax.legend(fontsize=12, loc='best')
fig.tight_layout()
# fig.savefig("modulewise_true_edges_angle.pdf")

### _3.3. Distance-based Sorting of Hits [Failed]_

- **Distance Method** doesn't work as one might remove an edge from inner layer to outer layers _i.e._ before and after the **skewed** layers gap.